In [190]:
import pandas as pd
import numpy as np
from scipy import stats

import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [191]:
train_df = pd.read_csv('C:/Users/User/Desktop/work_projects/brain_signals/5/action_imagery_near_one_person_train.csv', sep=';')
test_df = pd.read_csv('C:/Users/User/Desktop/work_projects/brain_signals/5/action_imagery_near_one_person_test.csv', sep=';')

In [192]:
train_df.head()

,time,condition,epoch,FC5,F3,FCz,F4,FC6,C5,C3,...,CP4,CP1,CP2,FC1,FC2,FC3,FC4,C1,C2,EMG
0,-1500,action,0,4.083370,5.773050,0.201995,2.521222,1.227479,2.879050,1.525725,...,-0.703621,-1.617210,-1.898605,1.542511,0.800545,2.453126,2.116559,-0.374185,-0.313328,700.354006
1,-1498,action,0,4.334879,5.940823,0.550814,3.056077,1.769989,3.052603,1.455476,...,-0.646888,-2.230537,-2.202046,1.818585,1.163469,2.443493,2.702282,-0.555758,-0.218421,798.403551
2,-1496,action,0,4.526513,6.035687,0.969258,3.652089,2.389096,3.180226,1.331029,...,-0.567853,-2.934153,-2.542141,2.123369,1.610394,2.411715,3.385222,-0.746074,-0.077012,940.262532
3,-1494,action,0,4.624738,6.031773,1.447590,4.271690,3.045508,3.238622,1.147808,...,-0.471721,-3.688103,-2.899911,2.440374,2.123954,2.356721,4.125346,-0.930178,0.109743,1106.261350
4,-1492,action,0,4.604022,5.912372,1.972501,4.873302,3.692196,3.210743,0.907327,...,-0.367698,-4.444057,-3.253814,2.752333,2.678072,2.280879,4.872123,-1.092541,0.335340,1254.080696


In [193]:
import pandas as pd

# Step 1: Read the CSV file (assuming train_df is already loaded)

# Step 2: Get the number of channels (i.e., the number of columns related to channels)
# The columns start from index 3 onward for the channel columns
channel_columns = train_df.columns[3:]

# Step 3: Create a list to hold the train_dfs_raw for each channel
train_dfs_raw = []

# Iterate through each channel column
for channel in channel_columns:
    # Group by 'epoch' and extract the time and channel values for each epoch
    channel_data = train_df[['time', 'epoch', 'condition', channel]]
    
    # Pivot the data: each epoch will become a row, and each time step will become a column
    channel_pivot = channel_data.pivot_table(index='epoch', columns='time', values=channel, aggfunc='first')
    
    # Optionally, include the 'condition' (action) column in the DataFrame
    # Use the first value of 'condition' for each epoch (this should be the same within an epoch)
    condition_data = train_df.groupby('epoch')['condition'].first()
    channel_pivot['class'] = condition_data  # Rename 'action' to 'class'
    
    # Replace 'active' with 1 and 'imagery' with 0 in the 'class' column
    channel_pivot['class'] = channel_pivot['class'].replace({'action': 1, 'imagery': 0})
    
    # Drop the first column (index 0) and reset the index to avoid keeping 'epoch' as index
    channel_pivot = channel_pivot.reset_index(drop=True)
    
    # Drop the row with the time ticks (which is typically the first row in the DataFrame)
    channel_pivot.columns.name = None  # Remove the extra column header name (time ticks)
    
    # Append this channel's dataframe to the list
    train_dfs_raw.append(channel_pivot)

# Step 4: Verify the result for the first channel's dataframe
train_dfs_raw[0].head()  # This will print the first few rows of the first channel's dataframe



C:\Users\User\AppData\Local\Temp\ipykernel_13464\2411653750.py:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  channel_pivot['class'] = channel_pivot['class'].replace({'action': 1, 'imagery': 0})
C:\Users\User\AppData\Local\Temp\ipykernel_13464\2411653750.py:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  channel_pivot['class'] = channel_pivot['class'].replace({'action': 1, 'imagery': 0})
C:\Users\User\AppData\Local\Temp\ipykernel_13464\2411653750.py:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will b

,-1500,-1498,-1496,-1494,-1492,-1490,-1488,-1486,-1484,-1482,...,1984,1986,1988,1990,1992,1994,1996,1998,2000,class
0,4.083370,4.334879,4.526513,4.624738,4.604022,4.450871,4.166431,3.767198,3.283626,2.756759,...,-0.066176,-0.093404,-0.134609,-0.193552,-0.271726,-0.367990,-0.478605,-0.597674,-0.717894,1
1,2.710441,3.119469,3.509298,3.833146,4.046041,4.110561,4.002028,3.712450,3.252610,2.651959,...,-4.210148,-4.961232,-5.339736,-5.378342,-5.136340,-4.691136,-4.128363,-3.531859,-2.974762,1
2,-1.371299,-1.078596,-0.707978,-0.280830,0.173373,0.620318,1.024946,1.355823,1.589151,1.711820,...,-0.658532,-0.673904,-0.728527,-0.828313,-0.975621,-1.168303,-1.399289,-1.656865,-1.925663,1
3,-0.573452,-0.755460,-0.997179,-1.274642,-1.552695,-1.788347,-1.935713,-1.952022,-1.803906,-1.473097,...,6.304868,5.926523,5.259975,4.361228,3.301716,2.161404,1.021172,-0.044665,-0.974881,1
4,0.745397,1.799175,2.991199,4.242298,5.461073,6.553612,7.433904,8.033592,8.309764,8.249763,...,5.312996,4.697758,3.991115,3.241260,2.495638,1.796909,1.179665,0.668155,0.275185,1


тест

In [194]:

# Assuming test_df is your test DataFrame without the 'condition' column# Step 2: Get the number of channels (i.e., the number of columns related to channels)
# The columns start from index 3 onward for the channel columns
channel_columns = test_df.columns[2:]

# Step 3: Create a list to hold the test_dfs_raw for each channel
test_dfs_raw = []

# Iterate through each channel column
for channel in channel_columns:
    # Group by 'epoch' and extract the time and channel values for each epoch
    channel_data = test_df[['time', 'epoch', channel]]
    
    # Pivot the data: each epoch will become a row, and each time step will become a column
    channel_pivot = channel_data.pivot_table(index='epoch', columns='time', values=channel, aggfunc='first')
    
    # If 'condition' column exists, add 'class' column (you can add your specific logic here)
    if 'condition' in test_df.columns:
        # Use the first value of 'condition' for each epoch (this should be the same within an epoch)
        condition_data = test_df.groupby('epoch')['condition'].first()
        channel_pivot['class'] = condition_data  # Rename 'action' to 'class'
        
        # Replace 'active' with 1 and 'imagery' with 0 in the 'class' column
        channel_pivot['class'] = channel_pivot['class'].replace({'action': 1, 'imagery': 0})
    
    # Drop the first column (index 0) and reset the index to avoid keeping 'epoch' as index
    channel_pivot = channel_pivot.reset_index(drop=True)
    
    # Drop the row with the time ticks (which is typically the first row in the DataFrame)
    channel_pivot.columns.name = None  # Remove the extra column header name (time ticks)
    
    # Append this channel's dataframe to the list
    test_dfs_raw.append(channel_pivot)

# Step 4: Verify the result for the first channel's dataframe
test_dfs_raw[0].head()  # This will print the first few rows of the first channel's dataframe



,-1500,-1498,-1496,-1494,-1492,-1490,-1488,-1486,-1484,-1482,...,1982,1984,1986,1988,1990,1992,1994,1996,1998,2000
0,2.907462,2.426750,1.875221,1.272268,0.639230,-0.002413,-0.632433,-1.232653,-1.787120,-2.281737,...,-19.818938,-18.802707,-17.210196,-15.117040,-12.631809,-9.889579,-7.042112,-4.245590,-1.647241,0.627555
1,2.802864,2.386150,1.837203,1.144311,0.302882,-0.681690,-1.791466,-2.994156,-4.242503,-5.475264,...,-3.832444,-5.811435,-7.361442,-8.500405,-9.270933,-9.732598,-9.953737,-10.003694,-9.946272,-9.834949
2,-1.456151,-1.435524,-1.367759,-1.247154,-1.072760,-0.848981,-0.585586,-0.297083,-0.001513,0.281223,...,2.888995,2.651608,2.476217,2.373774,2.342742,2.371097,2.439533,2.525336,2.606306,2.664143
3,4.871867,4.426501,3.920331,3.369144,2.789225,2.195908,1.602620,1.020543,0.458879,-0.074417,...,6.483555,6.479934,6.234236,5.787231,5.190877,4.503030,3.781968,3.081271,2.445578,1.907583
4,-4.247736,-3.830939,-3.258178,-2.541434,-1.707140,-0.794214,0.149555,1.072368,1.923931,2.660936,...,-5.695988,-5.567423,-5.299648,-4.894614,-4.366101,-3.739074,-3.047676,-2.332033,-1.634263,-0.994183


In [195]:
train_dfs_raw[0].head()

,-1500,-1498,-1496,-1494,-1492,-1490,-1488,-1486,-1484,-1482,...,1984,1986,1988,1990,1992,1994,1996,1998,2000,class
0,4.083370,4.334879,4.526513,4.624738,4.604022,4.450871,4.166431,3.767198,3.283626,2.756759,...,-0.066176,-0.093404,-0.134609,-0.193552,-0.271726,-0.367990,-0.478605,-0.597674,-0.717894,1
1,2.710441,3.119469,3.509298,3.833146,4.046041,4.110561,4.002028,3.712450,3.252610,2.651959,...,-4.210148,-4.961232,-5.339736,-5.378342,-5.136340,-4.691136,-4.128363,-3.531859,-2.974762,1
2,-1.371299,-1.078596,-0.707978,-0.280830,0.173373,0.620318,1.024946,1.355823,1.589151,1.711820,...,-0.658532,-0.673904,-0.728527,-0.828313,-0.975621,-1.168303,-1.399289,-1.656865,-1.925663,1
3,-0.573452,-0.755460,-0.997179,-1.274642,-1.552695,-1.788347,-1.935713,-1.952022,-1.803906,-1.473097,...,6.304868,5.926523,5.259975,4.361228,3.301716,2.161404,1.021172,-0.044665,-0.974881,1
4,0.745397,1.799175,2.991199,4.242298,5.461073,6.553612,7.433904,8.033592,8.309764,8.249763,...,5.312996,4.697758,3.991115,3.241260,2.495638,1.796909,1.179665,0.668155,0.275185,1


In [196]:
# train_dfs_raw[0].shape[1]
test_dfs_raw[0].shape[1]

1751

In [197]:
train_dfs_raw_cut = []

for df in train_dfs_raw:
    df = df.iloc[:, 500:-500]
    train_dfs_raw_cut.append(df)

train_df_raw_cut_0 = train_dfs_raw_cut[0]

тест

In [198]:
test_dfs_raw_cut = []

for df in test_dfs_raw:
    df = df.iloc[:, 500:-499]
    test_dfs_raw_cut.append(df)

test_df_raw_cut_0 = test_dfs_raw_cut[0]

In [199]:
# train_dfs_raw_cut[0].head()
train_dfs_raw_cut[0].shape[1]

752

In [200]:
window_size = 25

def smoothing_row(df, window_size):
    smoothed_df = df.apply(lambda row: row.rolling(window=window_size, min_periods=1).mean(), axis=1)
    return smoothed_df

train_dfs_raw_cut_smooth = [smoothing_row(df, window_size) for df in train_dfs_raw_cut]



тест

In [201]:
window_size = 25

def smoothing_row(df, window_size):
    smoothed_df = df.apply(lambda row: row.rolling(window=window_size, min_periods=1).mean(), axis=1)
    return smoothed_df

test_dfs_raw_cut_smooth = [smoothing_row(df, window_size) for df in test_dfs_raw_cut]


# Нейросеть 

In [202]:
# Convert each dataframe to a NumPy array and stack them
data_arrays = [df.values for df in train_dfs_raw_cut_smooth]  # List of arrays with shape [500, 150]
data = np.stack(data_arrays, axis=1)  # Shape: [500, 6, 150]

# Convert labels to NumPy array (ensure it's binary: 0 or 1)
labels = train_dfs_raw[0].iloc[:, -1].values  # Shape: [500]

In [203]:
labels

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [204]:
class SignalDataset(Dataset):
    def __init__(self, data, labels):
        """
        Args:
            data (numpy.ndarray): Array of shape [num_samples, channels, time_steps]
            labels (numpy.ndarray): Array of shape [num_samples]
        """
        self.data = torch.tensor(data, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [205]:
from torch.utils.data import DataLoader, Subset
import numpy as np
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.2, random_state=42, stratify=labels
)

# Create Dataset instances
train_dataset = SignalDataset(X_train, y_train)
test_dataset = SignalDataset(X_test, y_test)

# Now, let's balance the test dataset
# Get indices for each class
class_0_indices = np.where(y_test == 0)[0]
class_1_indices = np.where(y_test == 1)[0]

# We will take the minimum length of both classes to ensure balance
min_class_size = min(len(class_0_indices), len(class_1_indices))

# Select a random subset of indices from each class to balance the test dataset
class_0_sampled = np.random.choice(class_0_indices, min_class_size, replace=False)
class_1_sampled = np.random.choice(class_1_indices, min_class_size, replace=False)

# Combine the indices
balanced_indices = np.concatenate([class_0_sampled, class_1_sampled])

# Create a balanced test dataset
balanced_test_dataset = Subset(test_dataset, balanced_indices)

# Create DataLoader for the balanced test set
batch_size = 32
test_loader = DataLoader(balanced_test_dataset, batch_size=batch_size, shuffle=False)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [206]:
class CNN_LSTM_Model(nn.Module):
    def __init__(self, num_channels=29, time_steps=752, cnn_out_channels=128, 
                 cnn_kernel_size=3, lstm_hidden_size=128, lstm_num_layers=2, 
                 num_classes=2, dropout=0.5):
        super(CNN_LSTM_Model, self).__init__()
        
        # CNN component
        self.conv1 = nn.Conv1d(in_channels=num_channels, out_channels=cnn_out_channels, kernel_size=cnn_kernel_size, padding=1)
        self.bn1 = nn.BatchNorm1d(cnn_out_channels)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2)
        
        # Calculate the new time_steps after Conv and Pooling
        self.conv_output_time_steps = time_steps // 2  # After MaxPool with kernel_size=2
        
        # LSTM component
        self.lstm = nn.LSTM(input_size=cnn_out_channels, 
                            hidden_size=lstm_hidden_size, 
                            num_layers=lstm_num_layers, 
                            batch_first=True, 
                            dropout=dropout)
        
        # Fully connected layer
        self.fc = nn.Linear(lstm_hidden_size, num_classes)
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        """
        Args:
            x: Tensor of shape [batch_size, channels, time_steps]
        Returns:
            out: Tensor of shape [batch_size, num_classes]
        """
        # CNN layers
        x = self.conv1(x)  # [batch, cnn_out_channels, time_steps]
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool(x)  # [batch, cnn_out_channels, time_steps/2]
        
        # Prepare for LSTM: transpose to [batch, time_steps, features]
        x = x.permute(0, 2, 1)  # [batch, time_steps/2, cnn_out_channels]
        
        # LSTM layers
        lstm_out, (hn, cn) = self.lstm(x)  # lstm_out: [batch, time_steps/2, lstm_hidden_size]
        
        # Take the last time step's output
        out = lstm_out[:, -1, :]  # [batch, lstm_hidden_size]
        
        out = self.dropout(out)
        out = self.fc(out)  # [batch, num_classes]
        
        return out

In [207]:
import torch
import torch.nn as nn
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Assuming `labels` is already defined (e.g., the target labels for training)
# Convert labels to a numpy array if they are not already
labels = np.array(labels)

# Calculate class weights (use sklearn.utils.class_weight)
class_weights = compute_class_weight('balanced', classes=np.array([0, 1]), y=labels)
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

# Instantiate the model
model = CNN_LSTM_Model()
model.to(device)

# Use weighted loss function
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Define optimizer (Adam optimizer with a learning rate of 1e-3)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

Using device: cpu


In [208]:
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
import numpy as np

num_epochs = 10  # Adjust based on convergence
best_auc = 0.0
best_model_path = 'best_cnn_lstm_model_auc.pth'

# Outer loop with tqdm for epoch tracking
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    all_preds_prob_train = []  # Store predicted probabilities on training data
    all_targets_train = []  # Store true labels on training data
    
    # Use tqdm for batch-level progress
    with tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch') as batch_progress:
        for batch_idx, (signals, targets) in enumerate(batch_progress):
            signals = signals.to(device)  # [batch, 6, 150]
            targets = targets.to(device)  # [batch]
            
            # Forward pass
            outputs = model(signals)  # [batch, 2]
            loss = criterion(outputs, targets)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            # Store predicted probabilities (softmax outputs) for AUC calculation
            prob = torch.softmax(outputs, dim=1)  # [batch, 2]
            all_preds_prob_train.extend(prob.cpu().detach().numpy())  # Store in CPU for AUC calculation
            all_targets_train.extend(targets.cpu().numpy())  # Store true labels in CPU
            
            # Update tqdm with current loss for each batch
            batch_progress.set_postfix(loss=running_loss / (batch_idx + 1))
    
    avg_loss = running_loss / len(train_loader)

    # Calculate AUC on the training data (to track training progress)
    auc_train = roc_auc_score(all_targets_train, np.array(all_preds_prob_train)[:, 1])  # AUC for the positive class (class 1)
    
    # Print AUC and loss for the epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Train AUC: {auc_train:.4f}')
    
    # Evaluate on the test set for AUC after every epoch
    model.eval()
    all_preds_prob_test = []  # Store predicted probabilities on test data
    all_targets_test = []  # Store true labels on test data

    with torch.no_grad():
        for signals, targets in test_loader:
            signals = signals.to(device)
            targets = targets.to(device)
            
            outputs = model(signals)
            prob = torch.softmax(outputs, dim=1)  # [batch, 2]
            
            all_preds_prob_test.extend(prob.cpu().numpy())
            all_targets_test.extend(targets.cpu().numpy())
    
    # Calculate AUC on the test data
    auc_test = roc_auc_score(all_targets_test, np.array(all_preds_prob_test)[:, 1])
    print(f'Epoch [{epoch+1}/{num_epochs}], Test AUC: {auc_test:.4f}')
    
    # Save the model with the best AUC on the test set
    if auc_test > best_auc:
        best_auc = auc_test
        torch.save(model.state_dict(), best_model_path)
        print(f'Best model saved with AUC: {best_auc:.4f}')

# After training, load the best model and evaluate it on the test data
model.load_state_dict(torch.load(best_model_path))
model.eval()

# Final evaluation on the test set (for the record)
all_preds_prob_test_final = []
all_targets_test_final = []

with torch.no_grad():
    for signals, targets in test_loader:
        signals = signals.to(device)
        targets = targets.to(device)
        
        outputs = model(signals)
        prob = torch.softmax(outputs, dim=1)  # [batch, 2]
        
        all_preds_prob_test_final.extend(prob.cpu().numpy())
        all_targets_test_final.extend(targets.cpu().numpy())

# Calculate final AUC on the test data
final_auc_test = roc_auc_score(all_targets_test_final, np.array(all_preds_prob_test_final)[:, 1])
print(f'Final Test AUC: {final_auc_test:.4f}')

Epoch 1/10: 100%|██████████| 9/9 [00:17<00:00,  1.93s/batch, loss=0.616]


Epoch [1/10], Loss: 0.6161, Train AUC: 0.7393
Epoch [1/10], Test AUC: 0.8714
Best model saved with AUC: 0.8714


Epoch 2/10: 100%|██████████| 9/9 [00:16<00:00,  1.88s/batch, loss=0.497]


Epoch [2/10], Loss: 0.4969, Train AUC: 0.8633
Epoch [2/10], Test AUC: 0.8907
Best model saved with AUC: 0.8907


Epoch 3/10: 100%|██████████| 9/9 [00:17<00:00,  1.99s/batch, loss=0.492]


...

Epoch [8/10], Loss: 0.3737, Train AUC: 0.9248
Epoch [8/10], Test AUC: 0.9449


Epoch 9/10: 100%|██████████| 9/9 [00:12<00:00,  1.38s/batch, loss=0.327]


Epoch [9/10], Loss: 0.3265, Train AUC: 0.9201
Epoch [9/10], Test AUC: 0.9486


Epoch 10/10: 100%|██████████| 9/9 [00:09<00:00,  1.02s/batch, loss=0.378]
C:\Users\User\AppData\Local\Temp\ipykernel_13464\3264713664.py:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experiment

Epoch [10/10], Loss: 0.3780, Train AUC: 0.9179
Epoch [10/10], Test AUC: 0.9486
Final Test AUC: 0.9642


In [209]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Load the best model
model.load_state_dict(torch.load('best_cnn_lstm_model_auc.pth'))
model.eval()

# Final evaluation on the test set
all_preds = []
all_targets = []

with torch.no_grad():
    for signals, targets in test_loader:
        signals = signals.to(device)
        targets = targets.to(device)
        
        outputs = model(signals)
        _, preds = torch.max(outputs, 1)
        
        all_preds.extend(preds.cpu().numpy())
        all_targets.extend(targets.cpu().numpy())

# Calculate final accuracy
final_accuracy = accuracy_score(all_targets, all_preds)

# Calculate confusion matrix
final_cm = confusion_matrix(all_targets, all_preds)

# Extract TP, TN, FP, FN
TN, FP, FN, TP = final_cm.ravel()

# Print the accuracy and confusion matrix in a readable way
print(f'Final Test Accuracy: {final_accuracy:.4f}')
print(f'Final Confusion Matrix (readable format):')
print(f'True Negative (TN): {TN}')
print(f'False Positive (FP): {FP}')
print(f'False Negative (FN): {FN}')
print(f'True Positive (TP): {TP}')

# Classification Report (precision, recall, F1-score)
print("\nClassification Report:\n", classification_report(all_targets, all_preds))

# Optionally, you can print the confusion matrix in the usual format as well
print(f'\nConfusion Matrix (standard format):\n{final_cm}')

# Create the Comparison Table: True Labels vs Predicted Labels
comparison_df = pd.DataFrame({
    'True Label': all_targets,
    'Predicted Label': all_preds
})

print("\nComparison of True and Predicted Labels (first 10 rows):")
print(comparison_df.head(10))  # Display the first 10 rows for inspection


C:\Users\User\AppData\Local\Temp\ipykernel_13464\1752453437.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_cnn_lstm_model_auc.pth

Final Test Accuracy: 0.7424
Final Confusion Matrix (readable format):
True Negative (TN): 32
False Positive (FP): 1
False Negative (FN): 16
True Positive (TP): 17

Classification Report:
               precision    recall  f1-score   support

           0       0.67      0.97      0.79        33
           1       0.94      0.52      0.67        33

    accuracy                           0.74        66
   macro avg       0.81      0.74      0.73        66
weighted avg       0.81      0.74      0.73        66


Confusion Matrix (standard format):
[[32  1]
 [16 17]]

Comparison of True and Predicted Labels (first 10 rows):
   True Label  Predicted Label
0           0                0
1           0                0
2           0                0
3           0                0
4           0                0
5           0                0
6           0                0
7           0                0
8           0                0
9           0                0


# Предсказание

In [210]:
# Convert each dataframe in test_dfs_raw_smooth to a NumPy array and stack them
test_data_arrays = [df.values for df in test_dfs_raw_cut_smooth]  # List of arrays with shape [N, 150]
test_data = np.stack(test_data_arrays, axis=1)  # Shape: [N, 6, 150]

# Create a Dataset instance for test-only (without labels)
class TestOnlyDataset(Dataset):
    def __init__(self, data):
        """
        Args:
            data (numpy.ndarray): Array of shape [num_samples, channels, time_steps]
        """
        self.data = torch.tensor(data, dtype=torch.float32)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

# Create a DataLoader for `test_dfs_raw_smooth` (features only)
test_only_dataset = TestOnlyDataset(test_data)
batch_size = 32
test_only_loader = DataLoader(test_only_dataset, batch_size=batch_size, shuffle=False)

# Use the trained model to make predictions on the test data (features only)
model.eval()  # Set model to evaluation mode

all_preds = []

# Iterate over the test-only loader and make predictions
with torch.no_grad():
    for signals in test_only_loader:
        signals = signals.to(device)  # Move to GPU if needed
        
        outputs = model(signals)  # Get model output
        _, preds = torch.max(outputs, 1)  # Get predicted class (0 or 1)
        
        all_preds.extend(preds.cpu().numpy())  # Store predictions on CPU

# all_preds now contains all the predicted class labels (0 or 1)
print(f'Predicted classes for the test set: {all_preds}')


Predicted classes for the test set: [1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]


In [211]:
folder_path = 'C:/Users/User/Desktop/work_projects/brain_signals/5'

In [212]:
import os
# Ensure the folder exists, create it if necessary
os.makedirs(folder_path, exist_ok=True)


# Convert predictions to binary string
binary_string = ''.join([str(pred) for pred in all_preds])

# Define the full file path
file_path = os.path.join(folder_path, 'predictions.csv')

# Write the binary string to the CSV file without any headers or index
with open(file_path, 'w') as f:
    f.write(binary_string)

print(f"File saved to: {file_path}")

File saved to: C:/Users/User/Desktop/work_projects/brain_signals/5\predictions.csv


In [213]:
answer = pd.read_csv('C:/Users/User/Desktop/work_projects/brain_signals/5/predictions.csv')

In [214]:
answer

,100000101010001101000011000000100000


In [218]:
len(answer.iloc[0:0])

0